In [5]:
import pandas as pd, numpy as np, os, joblib, time, warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, roc_auc_score, confusion_matrix, classification_report
import lightgbm as lgb
DATA_PATH = "/content/preprocessed_earthquake_data.csv"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"{DATA_PATH} not found. Upload or give full path.")
df = pd.read_csv(DATA_PATH)
# detect target column
possible_targets = [c for c in df.columns if c.lower() in ("target","label","class","y")]
target_col = possible_targets[0] if possible_targets else df.columns[-1]
X, y = df.drop(columns=[target_col]), df[target_col]
# encode target if needed
if y.dtype == object or not pd.api.types.is_numeric_dtype(y):
    le = LabelEncoder(); y = le.fit_transform(y)
is_binary = len(np.unique(y)) == 2
# encode all object columns in X
X = X.copy()
for col in X.columns:
    if X[col].dtype == object or pd.api.types.is_categorical_dtype(X[col]):
        X[col] = LabelEncoder().fit_transform(X[col].astype(str))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scoring = 'f1' if is_binary else 'f1_macro'
lgb_model = lgb.LGBMClassifier(objective='binary' if is_binary else 'multiclass',
                               random_state=42, n_jobs=-1)
lgb_params = dict(
    num_leaves=[31,63],
    max_depth=[-1,5],
    learning_rate=[0.05,0.1],
    n_estimators=[100,200]
)
lgb_search = RandomizedSearchCV(lgb_model, lgb_params, n_iter=4,
                                scoring=scoring, cv=cv, random_state=42,
                                n_jobs=-1, verbose=1)
t0=time.time(); lgb_search.fit(X_train,y_train); print("LightGBM tuned in",round(time.time()-t0,2),"s")
svm_pipe = Pipeline([('scaler',StandardScaler()),('svc',SVC(probability=True,random_state=42))])
svm_params = dict(
    svc__C=[0.1,1],
    svc__kernel=['rbf'],
    svc__gamma=['scale','auto']
)
svm_search = RandomizedSearchCV(svm_pipe, svm_params, n_iter=2,
                                scoring=scoring, cv=cv, random_state=42,
                                n_jobs=-1, verbose=1)
t0=time.time(); svm_search.fit(X_train,y_train); print("SVM tuned in",round(time.time()-t0,2),"s")
def eval_model(model,name):
    y_pred = model.predict(X_test)
    try: y_proba = model.predict_proba(X_test)
    except: y_proba=None
    y_score = y_proba[:,1] if (y_proba is not None and is_binary) else None
    metrics = dict(
        model=name,
        accuracy=accuracy_score(y_test,y_pred),
        precision=precision_score(y_test,y_pred,average='binary' if is_binary else 'macro',zero_division=0),
        recall=recall_score(y_test,y_pred,average='binary' if is_binary else 'macro',zero_division=0),
        f1=f1_score(y_test,y_pred,average='binary' if is_binary else 'macro',zero_division=0),
        balanced_accuracy=balanced_accuracy_score(y_test,y_pred),
        roc_auc=roc_auc_score(y_test,y_score) if y_score is not None else np.nan
    )
    print(f"\n{name} classification report:\n",
          classification_report(y_test,y_pred,zero_division=0))
    print("Confusion matrix:\n",confusion_matrix(y_test,y_pred))
    return metrics
lgb_metrics = eval_model(lgb_search.best_estimator_,"LightGBM")
svm_metrics = eval_model(svm_search.best_estimator_,"SVM")
comp = pd.DataFrame([lgb_metrics,svm_metrics]).set_index('model').round(4)
print("\nComparison:\n",comp)
best = comp['f1'].idxmax()
print("\nBest model by F1:",best,"with F1=",comp.loc[best,'f1'
joblib.dump(lgb_search.best_estimator_,"lightgbm_model.pkl")
joblib.dump(svm_search.best_estimator_,"svm_pipeline.pkl")
comp.to_csv("model_comparison_summary.csv")
print("\nSaved: lightgbm_model.pkl, svm_pipeline.pkl, model_comparison_summary.csv")


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 16616, number of negative: 2111
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1139
[LightGBM] [Info] Number of data points in the train set: 18727, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.887275 -> initscore=2.063204
[LightGBM] [Info] Start training from score 2.063204
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh